In [ ]:
# Test Script
import pandas as pd
from shapely.geometry import shape
from shapely.ops import unary_union
import geopandas as gpd
from gerrychain.constraints import within_percent_of_ideal_population
from gerrychain.proposals import recom
from gerrychain.updaters import Tally, cut_edges
from functools import partial
from gerrychain import Graph, Partition, GeographicPartition, MarkovChain
from gerrychain.tree import recursive_tree_part
from gerrychain.accept import always_accept

def seawulf(working_directory, state, data, num_plans, num_districts):
    graph = Graph.from_file(working_directory + data)

    updaters = {
        "population": Tally("TOT_POP", alias="population"),
        "cut_edges": cut_edges,
        "democrats": Tally("PRSDEM01", alias="democrats"),
        "republicans": Tally("PRSREP01", alias="republicans"),
        "black": Tally("POP_BLK", alias="black"),
        "white": Tally("POP_WHT", alias="white"),
        "asian": Tally("POP_ASN", alias="asian"),
        "hispanic": Tally("POP_HISLAT", alias="hispanic"),
        "aindalk": Tally("POP_AINDALK", alias="aindalk"),
        "hipi": Tally("POP_HIPI", alias="hipi"),
        "other": Tally("POP_OTH", alias="other"),
        "twoormore": Tally("POP_TWOMOR", alias="twoormore"),
        "urban": Tally("Urban", alias="urban"),
        "suburban": Tally("Suburban", alias="suburban"),
        "rural": Tally("Rural", alias="rural"),
    }

    # Using The Existing Congressional District Plan
    # initial_partition = Partition(
    #     graph,
    #     assignment="CD_ID",
    #     updaters=updaters,
    # )
    
    # Using A Random Plan
    # Set the number of desired districts
    num_districts = num_districts

    # Calculate the total population and the target population for each district
    total_population = sum(graph.nodes[node]["TOT_POP"] for node in graph.nodes)
    ideal_population = total_population / num_districts
    epsilon = 0.05

    # Create a list of district labels
    district_labels = list(range(1, num_districts + 1))
    
    # Create the partition using recursive tree partitioning
    new_assignment = recursive_tree_part(
        graph,
        parts=district_labels,
        pop_target=ideal_population,
        pop_col="TOT_POP",
        epsilon=epsilon,
        node_repeats=1,
    )

    # Assign New Assignment to Intial Partion
    initial_partition = Partition(
        graph,
        assignment=new_assignment,
        updaters=updaters,
    )
    
    # Recom Algorithm
    proposal = partial(recom, pop_col="TOT_POP", pop_target=ideal_population, epsilon=0.05, node_repeats=2)
    
    # Population Constraint
    pop_constraint = within_percent_of_ideal_population(initial_partition, 0.05)

    # Number of District Plans in an Ensamble
    num_plans = num_plans

    # Gather all the Results
    all_results = []
    ensamble_measures = []
    
    # Run the Markov Chain Using the Recom Algorithm
    for plan in range(num_plans):
        chain = MarkovChain(
            proposal=proposal,
            constraints=pop_constraint,
            accept=always_accept,
            initial_state=initial_partition,
            total_steps= 2
        )
        
        # Final Results 
        final_results = {
            "district": [],
            "democrats": [],
            "republicans": [],
            "winner": [],
            "black_pct": [],
            "white_pct": [],
            "asian_pct": [],
            "hispanic_pct": [],
            "aindalk_pct": [],
            "hipi_pct": [],
            "other_pct": [],
            "twoormore_pct": [],
            "total_population": [],
            "category": [],
            "geometry": [],
        }     

        plan_summary = {
            "avg_income_difference": [],
            "avg_dem_support": [],
            "avg_rep_support": [],
            "rep_districts": [],
            "dem_districts": [],
            "rural_districts": [],
            "urban_districts": [],
            "suburban_districts": [],
            "poverty_districts": [],
            
        }

        statewide_average_income = 0
        
        # Calculate Measures
        if state == "CA":
            statewide_average_income = 95777.82
        elif state == "AL":
            statewide_average_income = 56036.51

        current_plan_income_difference = 0
        current_plan_dem_pct = 0
        current_plan_rep_pct = 0
        num_rep_districts = 0
        num_dem_districts = 0
        num_rural_districts = 0
        num_urban_districts = 0
        num_suburban_districts = 0
        num_poverty_districts = 0
                
        step_chain = list(chain)
        final_partition = step_chain[-1]
        
        for district in final_partition["democrats"].keys():
            dem_count = final_partition["democrats"][district]
            rep_count = final_partition["republicans"][district]
            black_count = final_partition["black"][district]
            white_count = final_partition["white"][district]
            asian_count = final_partition["asian"][district]
            hispanic_count = final_partition["hispanic"][district]
            aindalk_count = final_partition["aindalk"][district]
            hipi_count = final_partition["hipi"][district]
            other_count = final_partition["other"][district]
            twoormore_count = final_partition["twoormore"][district]
            total_population = final_partition["population"][district]
            
            # Store results
            # Election Data
            final_results["district"].append(district)
            final_results["democrats"].append(dem_count)
            final_results["republicans"].append(rep_count)
            
            if dem_count > rep_count:
                final_results["winner"].append("democrats")
                num_dem_districts += 1
            else:
                final_results["winner"].append("republicans")
                num_rep_districts += 1

            current_plan_dem_pct += (dem_count)/(dem_count + rep_count)
            current_plan_rep_pct += (rep_count)/(dem_count + rep_count)
        
            # Region Data
            urban_count = final_partition["urban"][district]
            suburban_count = final_partition["suburban"][district]
            rural_count = final_partition["rural"][district]

            region_category = []
            
            if urban_count >= suburban_count and urban_count >= rural_count:
                num_urban_districts += 1
                region_category.append("Urban")
            if suburban_count >= urban_count and suburban_count >= rural_count:
                num_suburban_districts += 1
                region_category.append("Suburban")
            if rural_count >= urban_count and rural_count >= suburban_count:
                num_rural_districts += 1
                region_category.append("Rural")

            final_results["category"].append(region_category)
            
            # Race Data
            final_results["black_pct"].append(black_count/total_population)
            final_results["white_pct"].append(white_count/total_population)
            final_results["asian_pct"].append(asian_count/total_population)
            final_results["hispanic_pct"].append(hispanic_count/total_population)
            final_results["aindalk_pct"].append(aindalk_count/total_population)
            final_results["hipi_pct"].append(hipi_count/total_population)
            final_results["other_pct"].append(other_count/total_population)
            final_results["twoormore_pct"].append(twoormore_count/total_population)
            final_results["total_population"].append(total_population)
    
        
            # Get the geometry for the precincts in this district and combine them
            precinct_geometries = []
            for precinct in final_partition.assignment:
                if final_partition.assignment[precinct] == district:
                    geometry = graph.nodes[precinct]["geometry"]
                    precinct_geometries.append(shape(geometry))  # Convert geometry to Shapely object
            
            # Combine the precinct geometries into a single geometry for the district
            combined_geometry = unary_union(precinct_geometries)  # Union all the precinct geometries
            
            # Add the combined geometry for the district
            final_results["geometry"].append(combined_geometry)

        plan_summary["avg_dem_support"].append(current_plan_dem_pct/num_districts)
        plan_summary["avg_rep_support"].append(current_plan_rep_pct/num_districts)
        plan_summary["rep_districts"].append(num_rep_districts)
        plan_summary["dem_districts"].append(num_dem_districts)
        plan_summary["rural_districts"].append(num_rural_districts)
        plan_summary["urban_districts"].append(num_urban_districts)
        plan_summary["suburban_districts"].append(num_suburban_districts)
        
        # income data
        
        eligible_precincts = graph.nodes(data=True)
        eligible_precincts = {k: v for k, v in eligible_precincts if v["TOT_HOUS21"] > 0.0}

        for district in final_partition["population"].keys():
            total_income = 0
            precinct_count = 0
    
            for precinct, attributes in eligible_precincts.items():
                if final_partition.assignment[precinct] == district:
                    total_income += attributes["MEDN_INC21"]
                    precinct_count += 1

            district_income = total_income / precinct_count if precinct_count > 0 else 0
            current_plan_income_difference += district_income - statewide_average_income
            
            if district_income <= 40000:
                num_poverty_districts += 1
                      
        all_results.append(pd.DataFrame(final_results))
        
        plan_summary["avg_income_difference"].append(current_plan_income_difference/num_plans)
        plan_summary["poverty_districts"].append(num_poverty_districts)
        
        ensamble_measures.append(pd.DataFrame(plan_summary))
    
    all_results_df = pd.concat(all_results, keys=range(num_plans), names=["plan_num", "index"])
    all_results_gdf = gpd.GeoDataFrame(all_results_df, geometry="geometry", crs="EPSG:4326")
    display(all_results_gdf)
    
    all_plan_summary = pd.concat(ensamble_measures, keys=range(num_plans), names=["plan_num", "index"])
    display(all_plan_summary)
    return all_results_gdf
    
        
    # results_df = pd.DataFrame(final_results)
    
    # # Print the results
    # print("Final District Results:")
    # print(results_df)
    
    # # Convert the list of results to a GeoDataFrame
    # gdf = gpd.GeoDataFrame(final_results, geometry='geometry', crs="EPSG:4326")
    
    # # Display the GeoDataFrame in a Jupyter notebook
    # display(gdf)
    
    # working_directory = "/Users/stanleymui/Downloads/CSE 416 Preprocessing Data/"
    # gdf.to_file(working_directory + "test.geojson", driver="GeoJSON")

if __name__ == "__main__":
    working_directory = "/Users/stanleymui/Downloads/CSE 416 Preprocessing Data/"
    california_data = "california_precinct_merged.geojson"
    all_results_gdf = seawulf(working_directory,"CA",california_data, 10, 10)